<a href="https://colab.research.google.com/github/rturquier/depythons/blob/main/Rendu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Projet informatique - Python pour le data scientist
##### Jérémie Stym-Popper, Luca Teodorescu, Rémi Turquier
# Les députés de l'opposition s'expriment-il différemment de ceux de la majorité ?

[Titre temporaire, c'est juste pour voir si ça marche]

In [2]:
# Imports
import pandas as pd


## Récupération des données
Les données ont été récupérées par l'intermédiaire de [l'API](https://github.com/regardscitoyens/nosdeputes.fr/blob/master/doc/api.md) mise à disposition par l'association Regards citoyens.

[à compléter]

## Nettoyage des données
Après une exploration préliminaire, nous avons choisi de nous concentrer sur deux groupes parlementaires. Les groupes LFI et LR nous ont paru ceux les plus susceptibles d'utiliser un discours différent et que nous pourrions distinguer dans notre modèle. 

In [152]:
# Import des données brutes récupérées avec l'API
data_url = "https://raw.githubusercontent.com/rturquier/depythons/main/Stock_csv/gd_inter.csv"
df_brut = pd.read_csv(data_url)
df_brut.sample(n=5)

,Unnamed: 0,groupe,nom,interventions
4,4,LFI,Bastien Lachaud,"["" C'est limpide. "", ' Ils le font déjà ! ', '..."
130,130,LR,Philippe Benassaya,"["" Non, mais je voudrais remercier monsieur le..."
35,35,LR,Ian Boucard,"["" Depuis le début de la séance, nos collègues..."
51,51,LR,Jean-Pierre Door,"[' Eh oui ! ', "" Cet été, le Gouvernement a dé..."
1,1,LFI,Muriel Ressiguier,"["" L'engagement de la procédure accélérée sur ..."


In [154]:
# Création d'une indicatrice `droite` qui sera la cible de la classification
df_brut = df_brut.\
            assign(droite = df_brut["groupe"] == "LR").\
            sort_values(by = ["droite"], ascending = False)
df_brut.head()

,Unnamed: 0,groupe,nom,interventions,droite
84,84,LR,Gilles Lurton,"["" Monsieur le Premier ministre, vous nous ave...",True
34,34,LR,Éric Ciotti,"["" J'entends votre argumentation. J'ai évoqué ...",True
22,22,LR,Philippe Gosselin,"[' Excellent rapport ! ', "" C'est très inquiét...",True
23,23,LR,Vincent Descoeur,"["" C'est vrai ! "", "" C'est une anomalie ! "", '...",True
86,86,LR,Jean-François Parigi,"[' Restez modeste ! ', ' Manipulation ! ', ' C...",True


In [155]:
# Équilibrage du nombre de députés
def balance_left_right(df):
  count = df.droite.value_counts()
  n_droite, n_gauche = count[True], count[False]
  df = df.sort_values(by = ["droite"], ascending = False)
  
  if n_droite > n_gauche :
    df = df[n_droite - n_gauche:]
  elif n_droite < n_gauche :
    df = df[2 * n_droite:]

  return df

df_brut = balance_left_right(df_brut)
df_brut.droite.value_counts()

True     51
False    51
Name: droite, dtype: int64

In [156]:
# Régler un problème de type
from ast import literal_eval
def convert_to_list(interventions):
  return literal_eval(str(interventions))

df_brut["interventions"] = df_brut["interventions"].apply(convert_to_list)
df_brut

,Unnamed: 0,groupe,nom,interventions,droite
68,68,LR,Jean-Marie Sermier,"[ Ce sont les départements qui payent ! , Pou...",True
71,71,LR,Laurence Trastour-Isnart,[ Il vise à supprimer la part salariale de la ...,True
66,66,LR,Marc Le Fur,"[Marc Le Fur Prochaine séance, demain, à neuf...",True
41,41,LR,Nicolas Forissier,[ Je ne vais pas reprendre les excellents argu...,True
39,39,LR,Jean-Jacques Gaultier,[ … mais contre votre gestion de la crise. Mai...,True
...,...,...,...,...,...
136,136,SOC,Marietta Karamanli,[ Je vous remercie de ces éléments et de votre...,False
137,137,SOC,Joël Aviragnet,"[ Il vise, lui aussi, à supprimer l'article in...",False
138,138,SOC,Boris Vallaud,[ Quelle incongruité que la majorité pénale so...,False
139,139,SOC,Alain David,[ Déposé par le groupe Socialistes et apparent...,False


In [161]:
# Séparer toutes les interventions en colonnes différentes
df_brut = df_brut.explode("interventions")

In [170]:
# Créer une feature "longeur de l'intervention"
df_brut = df_brut.assign(longeur = df_brut["interventions"].str.len())
df_brut

,Unnamed: 0,groupe,nom,interventions,droite,longeur
61,61,LR,Valérie Bazin-Malgras,Exactement ! Que faisons-nous donc ici ?,True,42
61,61,LR,Valérie Bazin-Malgras,Je rejoins les propos de mon collègue Larive ...,True,312
61,61,LR,Valérie Bazin-Malgras,Il a été rédigé par ma collègue Emmanuelle An...,True,200
60,60,LR,Nathalie Bassire,Les différents exposés de nos collègues ont b...,True,655
60,60,LR,Nathalie Bassire,Ce sous-amendement vise à compléter l'amendem...,True,651
...,...,...,...,...,...,...
156,156,SOC,George Pau-Langevin,"Avec cet amendement, nous voulons à nouveau e...",False,663
156,156,SOC,George Pau-Langevin,Nous appelons ici l'attention sur le fait que...,False,578
156,156,SOC,George Pau-Langevin,"Il vise à supprimer l'alinéa 9, qui rend poss...",False,660
156,156,SOC,George Pau-Langevin,Le groupe Socialistes et apparentés propose d...,False,421
